#  Install depences

In [ ]:
!apt-get update -y
#!apt-get upgrade -y
!apt install libgl1-mesa-glx -y
!pip install matplotlib
!pip install opencv-python
!/usr/bin/python -m pip install --upgrade pip


# Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import pathlib
import os
import datetime
import urllib.request
import cv2 
from random import seed
from random import randint
import tensorflow as tf 

image_dataset_folder="images_sets"
train_image_folder=os.path.join(os.path.join(image_dataset_folder,'images_train_set'))
test_image_folder=os.path.join(os.path.join(image_dataset_folder,'images_test_set'))
image_folder=test_image_folder

bugs_set_folder='insects_set'
train_bugs_folder=os.path.join(os.path.join(bugs_set_folder,'train_val_set'))
test_bugs_folder=os.path.join(os.path.join(bugs_set_folder,'test_set'))
dirt_folder=os.path.join(os.path.join(bugs_set_folder,'dirt_set'))
bugs_folder=test_bugs_folder


#!mkdir {image_folder}
seed(15)
print('Import libraries')


Import libraries


# Library

In [ ]:
# Create random position in Background image for paste image
def paste_image_area (background,pasted_image):
  y_background=randint(1,background.shape[0]-pasted_image.shape[0])
  x_background=randint(1,background.shape[1]-pasted_image.shape[1])
  return y_background,y_background+pasted_image.shape[0],x_background,x_background+pasted_image.shape[1]

#Check if pasted images are overlaped 
def is_it_overlay(overlay_array,image):
  for item_overlay_array in overlay_array:
    if any(e in range(item_overlay_array[0],item_overlay_array[1]) for e in range(image[0],image[1])) and any(e in range(item_overlay_array[2],item_overlay_array[3]) for e in range(image[2],image[3])) :
      return True
  return False   

def crop_from_contour(background,image,overlay_array,min_contourArea,overlay=True):
  #find contours from paste image
  ret,thresh = cv2.threshold(image, 110,255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU) 
  kernel = np.ones((5,5),np.uint8)
  
  #Find Contours from Threshold image
  cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2]
  for cnt in cnts:
    if cv2.contourArea(cnt) > min_contourArea:
      #print(cv2.contourArea(cnt))  
      break
  if cv2.contourArea(cnt) <= min_contourArea:
    return None
  
  if background.shape[0]<=image.shape[0] or background.shape[1]<=image.shape[1]:
    print(f'background.shape {background.shape} image.shape {image.shape}')
  #Create mask to paset pasted image on Background image
  mask = np.zeros(image.shape[:2],np.uint8)
  cv2.drawContours(mask, [cnt],-1, 255, -1)

  #Paste pasted image to Background image
  y0,y1,x0,x1=paste_image_area(background,image)
  if not is_it_overlay(overlay_array,(y0,y1,x0,x1)) or overlay==True: 
    overlay_array.append((y0,y1,x0,x1))
    background[y0:y1,x0:x1]=np.where(mask ==255, image, background[y0:y1,x0:x1])
    return background
  else:   
    #print('return none')
    return None 
  
#Random rotate in pasted image
def rotate_image(image,angle=0,pad=8):
  dim=(max(image.shape[:2])+pad,max(image.shape[:2])+pad)
  new_image =np.full(dim,255,dtype=np.uint8)
  
  y0 = int(.5 * new_image.shape[0]) - int(.5 * image.shape[0])
  x0 = int(.5 * new_image.shape[1]) - int(.5 * image.shape[1])
  y1 = int(.5 * new_image.shape[0]) + int(.5 * image.shape[0])
  x1 = int(.5 * new_image.shape[1]) + int(.5 * image.shape[1])
  if x1-x0<image.shape[1]:
    x1=image.shape[1]+x0
  if y1-y0<image.shape[0]:
    y1=image.shape[0]+y0  
  new_image[y0:y1,x0:x1]=image
   
  height, width = new_image.shape[:2]
  center = (width/2, height/2)
  rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=randint(1,359) if angle==0 else angle, scale=1)
  rot_im=cv2.warpAffine(src=new_image, M=rotate_matrix, dsize=(width,height))
  return np.where(rot_im==0,255,rot_im)

def image_scale(image,width_size=20):
  h,w=image.shape[0:2]
  scale=width_size/w
  image=cv2.resize(image,(int(w*scale),int(h*scale)), cv2.INTER_AREA)
  #print(f'width_size={width_size} w={w} scale={scale} {image is None} {image.shape}')
  return image

print('Init Functions')



# Create Image

## Create Background

In [ ]:
def create_background(max_num_of_dirts=4):
    original_img=cv2.cvtColor(cv2.imread(os.path.join(bugs_set_folder,'null_image.jpg')), cv2.COLOR_BGR2GRAY)
    dirt_images=os.listdir(dirt_folder)
    for x in range(randint(0,max_num_of_dirts)):
        min_contourArea=50
        name_of_dirt_image=os.path.join(dirt_folder,dirt_images[randint(0,len(dirt_images)-1)])
        #print(name_of_dirt_image)
        dirt_img=cv2.imread(name_of_dirt_image)
        dirt_img=cv2.cvtColor(dirt_img, cv2.COLOR_BGR2GRAY)
        
        h,w=dirt_img.shape[0:2]
        dirt_img=cv2.rectangle(dirt_img,(0,0),(w-1,h-1),(255,255,255),10) # really thick white rectangle
        
        dirt_img=image_scale(dirt_img,randint(20,40))
        
        dirt_img=rotate_image(dirt_img,0,0)
        
        #y0,y1,x0,x1=paste_image_area(original_img,dirt_img)
        #original_img[y0:y1,x0:x1]=np.where(dirt_img < 240, dirt_img, origina_img[y0:y1,x0:x1])
        min_contourArea=30
        last_image=original_img.copy()
        while True:
            
            overlay_array=[]
            #print(f' {last_image is  None}  {dirt_img is None}')
            #print(f' {last_image.shape}  {dirt_img.shape}')
            
            temp_original_img=crop_from_contour(last_image, dirt_img, overlay_array, min_contourArea,True)
            
            if  temp_original_img is not None:
                original_img=temp_original_img.copy()
                #print(f'Not None')
                break
            else:
                if min_contourArea-5>=0 :
                    min_contourArea=min_contourArea-5    
                    #print(f'min_contourArea {min_contourArea}')
                else:
                    #print('saddaadd')
                    break    

    return original_img

## Create image
## Create Dataset

In [ ]:
def create_image(background,bugs_folder,category_of_image):

  #print(f'Try image {category_of_image} insects')      
  #folder with bagus images
  bugs_images=os.listdir(bugs_folder)
  overlap=False
  #Array 
  overlay_array=[]
  image=None
  for i in range(category_of_image):
    count=0
    while True:
      count=count+1
      if category_of_image==0:
        #print(category_of_image)
        break
      else:  
        #Get file name of random bug from folder with bugs
        name_of_image=os.path.join(bugs_folder,bugs_images[randint(0,len(bugs_images)-1)])
        paste_img=cv2.imread(name_of_image)
        paste_img=cv2.cvtColor(paste_img, cv2.COLOR_BGR2GRAY)
        
        #Check if paste image is bigger than Background
        if background.shape[0]<=paste_img.shape[0] :
          if background.shape[1]<=paste_img.shape[1] :
            paste_img=image_scale(paste_img,int(paste_img.shape[1]*(background.shape[1]/paste_img.shape[1])))
          else:  
            paste_img=image_scale(paste_img,100)
        else:
          if background.shape[1]<=paste_img.shape[1] :
             paste_img=image_scale(paste_img,int(paste_img.shape[1]*(background.shape[1]/paste_img.shape[1])))
          
        #Random rotate paste image
        paste_img=rotate_image(paste_img,0,8)
          
        min_contourArea=200
        if count<2000:
          image=crop_from_contour(background,paste_img,overlay_array,min_contourArea,False)
        else:
          image=crop_from_contour(background,paste_img,overlay_array,min_contourArea,True)
          overlap=True
          count=0
        if image is  not None:
            #print(f'ok')
            break
  if category_of_image==0:
    #print(f'category_of_image={category_of_image}')
    image=background
    
  #print(f'Create image {category_of_image} insects overlap={overlap}')      
  return image,overlap         

def create_data_set(bugs_folder,image_folder,image_dataset_folder=image_dataset_folder,categories_of_images={0,1,2,3,4,5,6},number_of_images_per_category=200):
    !rm {image_folder}/*
    !rm insects_set/dirt_set/.ipynb_checkpoints -R
    #General Count of images
    count_of_image=0
    #General Count for images with overlap bagus
    count_overlap_image=0
    image=None
    for category_of_image in categories_of_images:
        for  number_of_image in range(number_of_images_per_category):
            while True:
              image,overlap=create_image(create_background(5),bugs_folder,category_of_image)
              if image is not None :
                break  
            if overlap==True:
                count_overlap_image=count_overlap_image+1  
            count_of_image= count_of_image + 1
            if not os.path.exists(image_folder):
              os.mkdir(image_folder)
            name_of_image_for_save = os.path.join(image_folder,f'insect_{count_of_image}_{category_of_image}.jpg')
            print(f'Save image {name_of_image_for_save}')
            #if image is None:
              #print('image none')
            cv2.imwrite(name_of_image_for_save,image)
    with open(os.path.join(image_dataset_folder,f'insects_{count_of_image}_test_new.txt'),'w') as f:
          f.write(f'count_of_image={count_of_image} count_overlap_image={count_overlap_image}  overlap {count_overlap_image/count_of_image*100}%') 
    print(f'count_of_image={count_of_image} count_overlap_image={count_overlap_image}  overlap {count_overlap_image/count_of_image*100}%')



In [ ]:
create_data_set(train_bugs_folder,train_image_folder,image_dataset_folder,{0,1,2,3,4,5,6},2000)
create_data_set(test_bugs_folder,test_image_folder,image_dataset_folder,{0,1,2,3,4,5,6},200)


In [ ]:
!rm insects_set/dirt_set/.ipynb_checkpoints -R
!rm images_sets/images_train_set/.ipynb_checkpoints -R
!rm images_sets/images_test_set/.ipynb_checkpoints -R
!tar cvzf images_sets/count_insects_dataset.tar.gz  images_sets/
